# Pre procesamiento

En este *notebook* se aplicará el pre-procesamiento a cada comentario de reddit. El resultado se guardará en un archivo que es similar al archivo origen, con la única diferencia que el comentario estará conformado por *strings* procesados.

Se realizan los siguientes pre-procesamientos:
1. Eliminación de *stop words*
2. Lematización utilizando Spacy
3. Eliminación de las palabras menos frecuentes
4. Conversión de los lemas a minúscula
5. Eliminación de palabras no alfanuméricas
6. Solo se consideran palabras cuyo *part-of-speech* son un nombre propio, un sustantivo o un pronombre. [Ver *Universal POS tags*](https://universaldependencies.org/docs/u/pos/)


In [1]:
import pandas as pd
import re, nltk, spacy, gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

TEXT_FILE_READ = 'docs/reddit_data.csv'	# Text to be processed
TEXT_SAVE_FILE = 'docs/preprocessing_reddit_data.csv'

In [3]:
nlp = spacy.load('es_core_news_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed
reddit = pd.read_csv(TEXT_FILE_READ)

In [4]:
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)


In [5]:
# Convert to list
data = reddit.body.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#print(data[:1])

['todo para decir que tapaste el baño. tira un balde de agua pa']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

[['todo', 'para', 'decir', 'que', 'tapaste', 'el', 'bano', 'tira', 'un', 'balde', 'de', 'agua', 'pa']]


In [11]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN', 'PROPN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] and not token.is_stop else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words)

#print(data_lemmatized[:2])


In [8]:
for index,row in enumerate(reddit['body']):
    reddit['body'][index] = data_lemmatized[index]

35134


<ipython-input-8-b81f95172c67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit['body'][index] = data_lemmatized[index]


In [10]:
reddit.to_csv(TEXT_SAVE_FILE, index=False)

